In [ ]:
# Homework 6 Template
# G. Besla & R. Li

In [7]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# my modules
from ReadFile import Read
# Step 1: modify CenterOfMass so that COM_P now takes a parameter specifying 
# by how much to decrease RMAX instead of a factor of 2
from CenterofMass import CenterOfMass



[-2.07  2.95 -1.45]
[ 0.94  6.32 -1.35] km / s


In [13]:
def OrbitCOM(galaxy, start, end, n=5): 
    """
    A function that loops over all the desired snapshots to compute the COM pos and vel as a function of time.
    
    inputs:
    galaxy : str
        The name of the galaxy (e.g., "MW", "M31", "M33").
    start : int
        The starting snapshot number.
    end : int
        The ending snapshot number.
    n : int, optional
        The interval at which snapshots are processed (default is 5).
    
    output:
    A text file with the time, x, y, z, vx, vy, and vz of the COM for the galaxy.
    """
    # Compose the filename for output
    fileout = f"Orbit_{galaxy}.txt"

    # Set tolerance and volDec for calculating COM_P in CenterOfMass
    delta = 0.1 
    volDec = 2 if galaxy != "M33" else 4  # Volume reduction factor for M33
    
    # Generate snapshot IDs to be processed
    snap_ids = np.arange(start, end + 1, n)
    if len(snap_ids) == 0:
        print("Error: No snapshots selected.")
        return
    
    # Initialize array to store orbital info: t, x, y, z, vx, vy, vz of COM
    orbit = np.zeros((len(snap_ids), 7))
    
    # Loop over selected snapshots
    for i, snap_id in enumerate(snap_ids):
        snap_str = f"{snap_id:03d}"  # Format snapshot number with leading zeros
        filename = f"/home/astr400b/VLowRes/{galaxy}_{snap_str}.txt"
        
        # Create a CenterOfMass object using disk particles (type 2)
        COM = CenterOfMass(filename, 2)
        
        # Compute the COM position and velocity
        COM_p = COM.COM_P(delta)
        COM_v = COM.COM_V(COM_p[0], COM_p[1], COM_p[2])
        
        # Store the results in the array
        orbit[i, 0] = COM.time / 1000  # Convert time to Gyr
        orbit[i, 1:4] = COM_p  # Store x, y, z positions
        orbit[i, 4:] = COM_v.value  # Store vx, vy, vz velocities
        
        # Print snap_id progress message
        print(f"Processed snapshot {snap_id}")
    
    # Save the computed orbit data to a text file
    np.savetxt(fileout, orbit, fmt = "%11.3f"*7, comments='#',
            header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                    .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
    print(f"Orbit saved to {fileout}")

In [12]:
# Recover the orbits and generate the COM files for each galaxy
# read in 800 snapshots in intervals of n=5
# Note: This might take a little while - test your code with a smaller number of snapshots first! 

# Load the computed center of mass data
MW_data = np.loadtxt("Orbit_MW.txt")    # MW
M31_data = np.loadtxt("Orbit_M31.txt")  # M31
M33_data = np.loadtxt("Orbit_M33.txt")  # M33

# Extract columns
time = MW_data[:, 0]  # Time in Gyr
MW_pos = MW_data[:, 1:4]  # x, y, z position
M31_pos = M31_data[:, 1:4]
M33_pos = M33_data[:, 1:4]
MW_vel = MW_data[:, 4:]   # vx, vy, vz velocity
M31_vel = M31_data[:, 4:]
M33_vel = M33_data[:, 4:]


FileNotFoundError: Orbit_MW.txt not found.

In [4]:
# Read in the data files for the orbits of each galaxy that you just created
# headers:  t, x, y, z, vx, vy, vz
# using np.genfromtxt
OrbitCOM("MW", 0, 800, 5)
OrbitCOM("M31", 0, 800, 5)
OrbitCOM("M33", 0, 800, 5)

NameError: name 'CenterOfMass' is not defined

In [ ]:
# You can use this function to return both the relative position and relative velocity for two 
# galaxies over the entire orbit  

# Compute the separation magnitude
sep_MW_M31 = np.linalg.norm(MW_pos - M31_pos, axis=1)  # MW - M31
sep_M33_M31 = np.linalg.norm(M33_pos - M31_pos, axis=1)  # M33 - M31


In [ ]:
# Determine the magnitude of the relative position and velocities 

# Compute the velocity magnitude
vel_MW_M31 = np.linalg.norm(MW_vel - M31_vel, axis=1)  # MW - M31
vel_M33_M31 = np.linalg.norm(M33_vel - M31_vel, axis=1)  # M33 - M31



In [ ]:
# Plot the Orbit of the galaxies 
#################################

# Set figure size
plt.figure(figsize=(12, 5))

# Plot Separation vs Time
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st plot
plt.plot(time, sep_MW_M31, label="MW - M31", color="blue")
plt.plot(time, sep_M33_M31, label="M33 - M31", color="red")
plt.xlabel("Time (Gyr)")
plt.ylabel("Separation (kpc)")
plt.title("Separation vs Time")
plt.legend()

In [ ]:
# Plot the orbital velocities of the galaxies 
#################################

# Plot Velocity vs Time
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd plot
plt.plot(time, vel_MW_M31, label="MW - M31", color="blue")
plt.plot(time, vel_M33_M31, label="M33 - M31", color="red")
plt.xlabel("Time (Gyr)")
plt.ylabel("Velocity (km/s)")
plt.title("Velocity vs Time")
plt.legend()

In [ ]:
# Show the plots
plt.tight_layout()
plt.show()